In [1]:
import os
import pandas as pd

In [2]:
path_pedalion = 'C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation'
path_gorman = 'C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/GORMAN/output_gorman_no_anomalies_aggiornati_per_lemmi_e_postag'

def load_pedalion(path_pedalion): #quali elementi presenti in pedalion sono presenti anche in gorman
    df_list = []
    for cartella in os.listdir(path_pedalion):
        folder_path = os.path.join(path_pedalion, cartella)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith('.csv'):
                    df = pd.read_csv(os.path.join(folder_path, file))
                    df = df[['@form', '@lemma', '@postag']]
                    df = df.rename(columns={
                        '@form': 'form',
                        '@lemma': 'lemmi',
                        '@postag': 'postag'
                    })
                    df_list.append(df)
    df_all = pd.concat(df_list, ignore_index=True)
    # Raggruppa e raccogli valori unici in lista
    return df_all.groupby('form').agg({
        'lemmi': lambda x: sorted(set(x.dropna())),
        'postag': lambda x: sorted(set(x.dropna()))
    }).reset_index().rename(columns={
        'lemmi': 'lemmi_pedalion',
        'postag': 'postag_pedalion'
    })

def load_gorman(path_gorman):
    df_list = []
    for cartella in os.listdir(path_gorman):
        folder_path = os.path.join(path_gorman, cartella)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                if file.endswith('.csv'):
                    df = pd.read_csv(os.path.join(folder_path, file), sep=';')
                    df = df[['@form', '@lemma', '@postag']]
                    df = df.rename(columns={
                        '@form': 'form',
                        '@lemma': 'lemmi',
                        '@postag': 'postag'
                    })
                    df_list.append(df)
    df_all = pd.concat(df_list, ignore_index=True)
    # Raggruppa e raccogli valori unici in lista
    return df_all.groupby('form').agg({
        'lemmi': lambda x: sorted(set(x.dropna())),
        'postag': lambda x: sorted(set(x.dropna()))
    }).reset_index().rename(columns={
        'lemmi': 'lemmi_gorman',
        'postag': 'postag_gorman'
    })


# Carica i dati
df_glaux = load_pedalion(path_pedalion)
df_gorman = load_gorman(path_gorman)

# Merge sulle 'form'
df_merged = pd.merge(df_glaux, df_gorman, on='form', how='inner')

# Visualizza o salva
# df_merged.to_csv('output_confronto.csv', sep=';', index=False)

In [3]:

df_merged_no_punct = df_merged[~df_merged['lemmi_gorman'].apply(lambda x: 'punc' in x)]

print(df_merged_no_punct)

            form lemmi_pedalion         postag_pedalion  lemmi_gorman  \
4            -δὲ           [δέ]  [b--------, d--------]          [δέ]   
5            -τε           [τε]             [b--------]          [τε]   
10        Αἰγίνῃ       [Αἴγινα]             [n-s---fd-]      [Αἴγινα]   
11     Αἰγυπτίαν     [Αἰγυπτία]             [a-s---fa-]   [Αἰγύπτιος]   
12     Αἰγυπτίας     [Αἰγυπτία]             [a-s---fg-]   [Αἰγύπτιος]   
...          ...            ...                     ...           ...   
16035       Ῥόδῳ        [Ῥόδος]             [n-s---fd-]       [Ῥόδος]   
16036       Ῥώμη         [Ῥώμη]  [n-s---fn-, n-s---fv-]  [ῥώμη, Ῥώμη]   
16037      Ῥώμην         [Ῥώμη]             [n-s---fa-]  [ῥώμη, Ῥώμη]   
16038      Ῥώμης         [Ῥώμη]             [n-s---fg-]  [ῥώμη, Ῥώμη]   
16039       Ῥώμῃ         [Ῥώμη]             [n-s---fd-]  [ῥώμη, Ῥώμη]   

                postag_gorman  
4      [c--------, d--------]  
5      [c--------, d--------]  
10                [n-s---fd

In [4]:
def percentuale_match(lista1, lista2):
    """Restituisce la percentuale di elementi di lista1 che sono anche in lista2."""
    if not lista1:  # evita divisione per zero
        return 0.0
    match = len(set(lista1) & set(lista2))
    return match / len(lista1)

# Aggiungi le nuove colonne al DataFrame mettere i nomi giusti delle colonne
df_merged_no_punct["percentuale_match_lemmi"] = df_merged_no_punct.apply(
    lambda row: percentuale_match(row["lemmi_gorman"], row["lemmi_pedalion"]),
    axis=1
)

df_merged_no_punct["percentuale_match_postag"] = df_merged_no_punct.apply(
    lambda row: percentuale_match(row["postag_gorman"], row["postag_pedalion"]),
    axis=1
)


C:\Users\lcolo\AppData\Local\Temp\ipykernel_24220\1453740313.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_no_punct["percentuale_match_lemmi"] = df_merged_no_punct.apply(
C:\Users\lcolo\AppData\Local\Temp\ipykernel_24220\1453740313.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_no_punct["percentuale_match_postag"] = df_merged_no_punct.apply(


In [5]:
df_merged_no_punct

,form,lemmi_pedalion,postag_pedalion,lemmi_gorman,postag_gorman,percentuale_match_lemmi,percentuale_match_postag
4,-δὲ,[δέ],"[b--------, d--------]",[δέ],"[c--------, d--------]",1.0,0.5
5,-τε,[τε],[b--------],[τε],"[c--------, d--------]",1.0,0.0
10,Αἰγίνῃ,[Αἴγινα],[n-s---fd-],[Αἴγινα],[n-s---fd-],1.0,1.0
11,Αἰγυπτίαν,[Αἰγυπτία],[a-s---fa-],[Αἰγύπτιος],[a-s---fa-],0.0,1.0
12,Αἰγυπτίας,[Αἰγυπτία],[a-s---fg-],[Αἰγύπτιος],"[a-p---fa-, a-s---fg-]",0.0,0.5
...,...,...,...,...,...,...,...
16035,Ῥόδῳ,[Ῥόδος],[n-s---fd-],[Ῥόδος],[n-s---fd-],1.0,1.0
16036,Ῥώμη,[Ῥώμη],"[n-s---fn-, n-s---fv-]","[ῥώμη, Ῥώμη]",[n-s---fn-],0.5,1.0
16037,Ῥώμην,[Ῥώμη],[n-s---fa-],"[ῥώμη, Ῥώμη]",[n-s---fa-],0.5,1.0
16038,Ῥώμης,[Ῥώμη],[n-s---fg-],"[ῥώμη, Ῥώμη]",[n-s---fg-],0.5,1.0


In [6]:
# Calcolo dei match booleani (almeno un match = percentuale > 0)
df_merged_no_punct["match_lemmi_bool"] = df_merged_no_punct["percentuale_match_lemmi"] > 0
df_merged_no_punct["match_postag_bool"] = df_merged_no_punct["percentuale_match_postag"] > 0

# Percentuali globali
perc_lemmi = df_merged_no_punct["match_lemmi_bool"].mean() * 100
perc_postag = df_merged_no_punct["match_postag_bool"].mean() * 100

# Stampa dei risultati
print(f"Percentuale di match nei lemmi: {perc_lemmi:.2f}%")
print(f"Percentuale di match nei postag: {perc_postag:.2f}%")

Percentuale di match nei lemmi: 95.78%
Percentuale di match nei postag: 92.26%


C:\Users\lcolo\AppData\Local\Temp\ipykernel_24220\2393955951.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_no_punct["match_lemmi_bool"] = df_merged_no_punct["percentuale_match_lemmi"] > 0
C:\Users\lcolo\AppData\Local\Temp\ipykernel_24220\2393955951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_no_punct["match_postag_bool"] = df_merged_no_punct["percentuale_match_postag"] > 0


## Uniformità Dataset Pedalion

In [7]:
import os
import pandas as pd

In [13]:
FOLDER = 'C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation' #path pedalion!
mappa_forme = {}

for root, dirs, files in os.walk(FOLDER):
    print("analysed dir", root)
    for file in files:
        if file.endswith('.csv'):
            csv_path = os.path.join(root, file)
            df = pd.read_csv(csv_path, sep=',')

            for idx, row in df.iterrows():
                forma = str(row.get('@form', '')).strip()
                lemma = str(row.get('@lemma', '')).strip()
                postag = str(row.get('@postag', '')).strip()

                if forma not in mappa_forme:
                    mappa_forme[forma] = {
                        "lemmi": set(),
                        "postag": set()
                    }

                mappa_forme[forma]["lemmi"].add(lemma)
                mappa_forme[forma]["postag"].add(postag)



analysed dir C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation
analysed dir C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation\aeneas.xml
analysed dir C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation\aesop1.xml
analysed dir C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation\charb1.xml
analysed dir C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation\chion.xml
analysed dir C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation\crit.xml
analysed dir C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation\epictetus.xml
analysed dir C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/PEDALION/output_pedalion_con_relation\epicurus1.xml
analysed dir C:/Users/lcolo/Desktop/TESI/CODICI 

In [14]:
# Calcola le medie
total_lemmi = sum(len(entry["lemmi"]) for entry in mappa_forme.values())
total_postag = sum(len(entry["postag"]) for entry in mappa_forme.values())
total_forme = len(mappa_forme)

media_lemmi_per_forma = total_lemmi / total_forme if total_forme else 0
media_postag_per_forma = total_postag / total_forme if total_forme else 0

print(f"Numero totale forme diverse: {total_forme}")
print(f"Media lemmi per forma: {media_lemmi_per_forma:.2f}")
print(f"Media postag per forma: {media_postag_per_forma:.2f}")

Numero totale forme diverse: 33634
Media lemmi per forma: 1.03
Media postag per forma: 1.10


In [9]:
records = []
for forma, dati in mappa_forme.items():
    records.append({
        "form": forma,
        "lemmi": sorted(list(dati["lemmi"])),
        "postag": sorted(list(dati["postag"])),
        "conteggio_lemmi": len(dati["lemmi"]),
        "conteggio_postag": len(dati["postag"]),
    })

df_finale = pd.DataFrame(records)
df_finale.to_csv('uniformita_dataset_pedalion_con_relation.csv', sep=';')

In [10]:
df_finale

,form,lemmi,postag,conteggio_lemmi,conteggio_postag
0,ὅσοις,[ὅσος],"[p-p---md-, p-p---nd-]",1,2
1,τῶν,"[ὁ, ὅς]","[l-p---fg-, l-p---mg-, l-p---ng-, p-p---mg-, p...",2,5
2,ἀνθρώπων,[ἄνθρωπος],[n-p---mg-],1,1
3,ἐκ,[ἐκ],[r--------],1,1
4,τῆς,[ὁ],[l-s---fg-],1,1
...,...,...,...,...,...
33629,εὐδαίμονας,[εὐδαίμων],[a-p---ma-],1,1
33630,ἡγῆται,[ἡγέομαι],[v3spse---],1,1
33631,οἶδά,[οἶδα],[v1sria---],1,1
33632,ἐπεθύμησας,[ἐπιθυμέω],[v2saia---],1,1


# GLAUX 

In [11]:
import os
import pandas as pd

FOLDER = 'C:/Users/lcolo/Desktop/TESI/CODICI DEFINITIVI DIZ UNICO-2/GLAUX/output_glaux'

mappa_forme = {}

# Scorre tutti i file nella cartella piatta
for file in os.listdir(FOLDER):
    if file.endswith('.xml.csv'):
        csv_path = os.path.join(FOLDER, file)
        df = pd.read_csv(csv_path, sep=',')

        for idx, row in df.iterrows():
            forma = str(row.get('@form', '')).strip()
            lemma = str(row.get('@lemma', '')).strip()
            postag = str(row.get('@postag', '')).strip()

            if forma not in mappa_forme:
                mappa_forme[forma] = {
                    "lemmi": set(),
                    "postag": set()
                }

            mappa_forme[forma]["lemmi"].add(lemma)
            mappa_forme[forma]["postag"].add(postag)

# Calcola le medie
total_lemmi = sum(len(entry["lemmi"]) for entry in mappa_forme.values())
total_postag = sum(len(entry["postag"]) for entry in mappa_forme.values())
total_forme = len(mappa_forme)

media_lemmi_per_forma = total_lemmi / total_forme if total_forme else 0
media_postag_per_forma = total_postag / total_forme if total_forme else 0

print(f"Numero totale forme diverse: {total_forme}")
print(f"Media lemmi per forma: {media_lemmi_per_forma:.2f}")
print(f"Media postag per forma: {media_postag_per_forma:.2f}")


Numero totale forme diverse: 633653
Media lemmi per forma: 1.02
Media postag per forma: 1.15
